In [11]:
import numpy as np
import copy
import json
import math
from glob import glob
import scipy.spatial.distance as sciDist
from tqdm import tqdm
import requests
import time
import itertools
import random
import os
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from itertools import islice
from PIL import Image
import re
from tqdm import tqdm
from path_decomposition import linkMajor, computeSolSteps, linkMajor2B


# Headless simulator version
index = 0 # local server index 
API_ENDPOINT = 'http://localhost:4001/simulation-8bar' # NOT THE LS VERSION
HEADERS = {"Content-Type": "application/json"}
batchCount = 1 # Send this number of samples to MotionGen each time 
speedscale = 1
steps = 360
minsteps = int(steps*20/360)

mechType = 3

topo_numbers = []
with open('8bar.txt', 'r') as f:
    for line in f:
        if line.startswith('Topo '):
            parts = line.strip().split()
            if len(parts) >= 2 and parts[1].isdigit() and len(parts[1]) == 3:
                topo_numbers.append(parts[1])
print(topo_numbers)
# ['811', '812', '813', '814', '815', '816', '817', '818', '819', '821', '822', '823', '824', '825', '831', '832']

typesList = [f"Type{num}-" for num in topo_numbers]
print(typesList)
#  0            1           2            3          4            5          6          7             8           9           10           11         12          13           14         15 
# ['Type811-', 'Type812-', 'Type813-', 'Type814-', 'Type815-', 'Type816-', 'Type817-', 'Type818-', 'Type819-', 'Type821-', 'Type822-', 'Type823-', 'Type824-', 'Type825-', 'Type831-', 'Type832-']

# shape of init pos 
types   = [typesList[mechType]]
initPos = [22] #11 pts, 22 coords
randSeed= [44] #ignore
couplerCurveIndices = [10]
distLens = [55] #ignore

output_dir = "outputs-8bar"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for t in typesList:
    dir_path = os.path.join(output_dir, t + "0")
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

['811', '812', '813', '814', '815', '816', '817', '818', '819', '821', '822', '823', '824', '825', '831', '832']
['Type811-', 'Type812-', 'Type813-', 'Type814-', 'Type815-', 'Type816-', 'Type817-', 'Type818-', 'Type819-', 'Type821-', 'Type822-', 'Type823-', 'Type824-', 'Type825-', 'Type831-', 'Type832-']


In [36]:
# good old ones 

def isValid(seq):
    if len(seq.shape) == 2:
        isVal = np.var(seq[:,0]) <= 5e-3 and np.var(seq[:,1]) <= 5e-3
    else:
        isVal = len(seq) == 0 or np.var(seq) <= 5e-3

    if isVal:
        return False
    else:
        return True

def center_data(X):
    """ Center the data by subtracting the mean of each column.
        Return the translated X and the translation matrix 
    """
    m = np.mean(X, axis=0) # (n, 2)
    return X - m, np.matrix([[1, 0, -m[0]], [0, 1, -m[1]], [0, 0, 1]]) # equal to XP this is a translation matrix tranposed  


def scale_data(X, scaling = 0): 
    """ Scale the data according to two different metrics 
        If scaling == 0 (default), scaling method is normalization (average distance 1)
        If otherwise, scaling method is standardization to a certain scale 
        Return the scaled X, and the scaling matrix. 
    """
    if scaling == 0:
        # use variance. 
        denom = np.sqrt(np.var(X[:,0]) + np.var(X[:,1]))
        scaled_curve = X /denom
        ScaleMat = np.matrix([[1/denom, 0, 0], [0, 1/denom, 0], [0, 0, 1]])
    else:
        # Compute the maximum distance from the origin 
        max_distance = np.max(np.linalg.norm(X, axis=1))
        scaled_curve = X * scaling / max_distance
        ScaleMat = np.matrix([[scaling/max_distance, 0, 0], [0, scaling/max_distance, 0], [0, 0, 1]])
    return scaled_curve, ScaleMat


def rotate_data(X, tol = 1e-4, randinit = False): 
    """ Performs the PCA and determines rotation angle phi 
        More precisely it is snapping the greatest principal axis to the X-axis. 
        Return the rotated X and the rotation matrix 
    """
    # Ensure input is numpy array
    X = np.array(X)
    
    phiInit = 0
    if randinit:
        phiInit = np.random.rand() * math.pi * 2 

    rotationMatInit = np.matrix([
        [np.cos(phiInit), -np.sin(phiInit), 0], 
        [np.sin(phiInit), np.cos(phiInit), 0],
        [0, 0, 1] 
    ])

    X0 = rotate_curve(X, phiInit)
    
    # CRITICAL FIX: Ensure X0 is a numpy array, not a tuple
    X0 = np.array(X0)
    
    # Now these operations will work properly
    cx = np.mean(X0[:,0])
    cy = np.mean(X0[:,1])
    covar_xx = np.sum((X0[:,0] - cx)*(X0[:,0] - cx))/X0.shape[0]
    covar_xy = np.sum((X0[:,0] - cx)*(X0[:,1] - cy))/X0.shape[0]
    covar_yx = np.sum((X0[:,1] - cy)*(X0[:,0] - cx))/X0.shape[0]
    covar_yy = np.sum((X0[:,1] - cy)*(X0[:,1] - cy))/X0.shape[0]
    covar = np.array([[covar_xx, covar_xy],[covar_yx, covar_yy]])

    if np.abs(np.linalg.det(covar)) < tol:
        phi = 0 # why rotate anyway? 
    else:
        eig_val, eig_vec= np.linalg.eig(covar) 
        # Inclination of major principal axis w.r.t. x axis
        # Enforcing the cross-product of the two eigenvectors to be greater than 0. 
        # Not necessary, but it looks clean to do so. 
        # Eigenvector matrix: [a, b], det = crossproduct of b x a
        if np.linalg.det(eig_vec) > 0:
            eig_vec[0,:] = -eig_vec[0,:] # enforcing a x b > 0 
        if eig_val[0] > eig_val[1]:
            phi= np.arctan2(eig_vec[1,0], eig_vec[0,0])
        else:
            phi= np.arctan2(eig_vec[1,1], eig_vec[0,1])
    
    rotated_curve = rotate_curve(X0, phi)
    
    # Ensure the final output is also a numpy array
    rotated_curve = np.array(rotated_curve)
    
    rotationMat = np.matrix([
        [np.cos(phi), -np.sin(phi), 0], 
        [np.sin(phi), np.cos(phi), 0],
        [0, 0, 1] 
    ])

    return rotated_curve, np.matmul(rotationMat, rotationMatInit)


def reflect_data(X):
    """ Computes the third order moment and determines the reflections 
        The data must be rotated before this step. 

    """
    # Reflection normalization 
    x_scaled = X[:, 0]
    y_scaled = X[:, 1]

    # see paper Geometric Invariant Curve and Surface Normalization
    # compute the 3rd-order moments 
    m12 = np.sum((x_scaled**1)*(y_scaled**2))
    m21 = np.sum((x_scaled**2)*(y_scaled**1))
    signm12 = np.sign(m12)
    signm21 = np.sign(m21)
    if np.abs(signm12) < 1e-5:
        signm12 = 1
    if np.abs(signm21) < 1e-5:
        signm21 = 1

    reflectionMat = np.array(
        [[signm12, 0],
         [0, signm21]]
    ) 

    if np.abs(m12) > np.abs(m21):
        reflectionMat = np.matmul(np.array([[0,1],[1,0]]), reflectionMat)

    reflected_Curve = np.matmul(reflectionMat, np.array(X).T).T
    reflectionMat = np.matrix(
        [[reflectionMat[0,0], reflectionMat[0,1], 0], 
         [reflectionMat[1,0], reflectionMat[1,1], 0], 
         [0, 0, 1]
        ]
    ) 

    return reflected_Curve, reflectionMat




def get_pca_inclination(qx, qy, ax=None, label=''):
    """ Performs the PCA
        Return transformation matrix
    """
    cx = np.mean(qx)
    cy = np.mean(qy)
    covar_xx = np.sum((qx - cx)*(qx - cx))/len(qx)
    covar_xy = np.sum((qx - cx)*(qy - cy))/len(qx)
    covar_yx = np.sum((qy - cy)*(qx - cx))/len(qx)
    covar_yy = np.sum((qy - cy)*(qy - cy))/len(qx)
    covar = np.array([[covar_xx, covar_xy],[covar_yx, covar_yy]])
    eig_val, eig_vec= np.linalg.eig(covar)

    # Inclination of major principal axis w.r.t. x axis
    if eig_val[0] > eig_val[1]:
        phi= np.arctan2(eig_vec[1,0], eig_vec[0,0])
    else:
        phi= np.arctan2(eig_vec[1,1], eig_vec[0,1])

    return phi


def get_normalize_curve(jd, steps=None, rotations=1, normalize=True, transformParas=None):
    jd = np.array(jd)
    joint_data_n, x_mean, y_mean, denom, phi = [], None, None, None, None
    if isValid(jd):
        if steps:
            sample_indices = np.linspace(0, jd.shape[0]-1, steps, dtype=np.int32)
            jd = jd[sample_indices,:]
        if normalize:
            if not transformParas:
                x_mean = np.mean(jd[:,0], axis=0, keepdims=True)
                y_mean = np.mean(jd[:,1], axis=0, keepdims=True)
            else:
                x_mean, y_mean, denom, phi = transformParas
            jd[:,0] = jd[:,0] - x_mean
            jd[:,1] = jd[:,1] - y_mean

            if not transformParas:
                denom = np.sqrt(np.var(jd[:,0], axis=0, keepdims=True) + np.var(jd[:,1], axis=0, keepdims=True))
                denom = np.expand_dims(denom, axis=1)
            jd = jd / denom
            t = 0
        if not transformParas:
            phi = -get_pca_inclination(jd[:,0], jd[:,1])
        jd[:,0], jd[:, 1] = rotate_curve(jd, phi)
        for tt in range(rotations):
            joint_data_n.append(jd.copy())
            if rotations > 1:
                jd[:,0], jd[:,1] = rotate_curve(jd, t)
                t = 2*np.pi/rotations

    return joint_data_n, x_mean, y_mean, denom, phi



    
def rotate_curve(cur, theta):
    cpx = cur[:,0]*np.cos(theta) - cur[:,1]*np.sin(theta)
    cpy = cur[:,0]*np.sin(theta) + cur[:,1]*np.cos(theta)
    return cpx, cpy


def digitize_seq(nums, minlim, maxlim, bin_size=64):
    bins = np.linspace(minlim, maxlim, bin_size-1)
    nums_indices = np.digitize(nums, bins)
    return nums_indices


def get_normalize_joint_data_wrt_one_curve(joint_data, ref_ind = 4):
    ''' input s = [num_curves, num_points, 2]
    '''
    joint_data_n = []
    s = np.array(joint_data)

    if isValid(s[ref_ind]):
        x_mean = np.mean(s[ref_ind:ref_ind+1,:,0], axis=1, keepdims=True)
        y_mean = np.mean(s[ref_ind:ref_ind+1,:,1], axis=1, keepdims=True)
        s[:,:,0] = s[:,:,0] - x_mean
        s[:,:,1] = s[:,:,1] - y_mean
        denom = np.sqrt(np.var(s[ref_ind:ref_ind+1,:,0], axis=1, keepdims=True) + np.var(s[ref_ind:ref_ind+1,:,1], axis=1, keepdims=True))
        denom = np.expand_dims(denom, axis=2) #is this scale? 
        s = s / denom
        phi = -get_pca_inclination(s[ref_ind:ref_ind+1,:,0], s[ref_ind:ref_ind+1,:,1])
        for i in range(s.shape[0]):
            s[i,:,0], s[i,:,1] = rotate_curve(s[i], phi)
    else:
        return s, [None, None, None, None], False

    # s has a shape of (j_num, state, dim)
    return s, [x_mean[0][0], y_mean[0][0], denom[0][0][0], phi], True # tx, ty, scaling, rotation angle 


##############################################################################################
# There are some other necessary transformations. (x_mean, y_mean, phi, denom) are from get_normalize_curve. 
##############################################################################################
def get_image_from_point_cloud(points, xylim, im_size, inverted = True, label=None):
    mat = np.zeros((im_size, im_size, 1), dtype=np.uint8)
    x = digitize_seq(points[:,0], -xylim, xylim, im_size)
    if inverted:
        y = digitize_seq(points[:,1]*-1, -xylim, xylim, im_size)
        mat[y, x, 0] = 1
    else:
        y = digitize_seq(points[:,1], -xylim, xylim, im_size)
        mat[x, y, 0] = 1
    return mat


def process_mech_102723(jointData, ref_ind, im_size = 64, xylim = 3.5, inverted = True, swapAxes = True):
    paras = None

    # It is possible the jointData format is (angles, joint, (x, y)). 
    # You should put a True if this happens. (This is how files are saved).
    # I literally don't understand why I saved jointData with a shape of (angles, joint, (x, y)) 
    if swapAxes:
        jointData = np.swapaxes(jointData, 0, 1)

    # This converts all 
    jointData, paras, success = get_normalize_joint_data_wrt_one_curve(jointData, ref_ind= ref_ind)

    # jointData format from now on becomes np.array with a shape of (joint, curve_length, dimension)
    jointData = np.array(jointData)

    if success:
        # get binaryImage 
        jd = jointData[ref_ind]
        mat = get_image_from_point_cloud(jd, xylim=xylim, im_size=im_size, inverted=inverted)
        return mat, paras, success
    else: 
        return None, None, success

def calc_dist(coord):
    # Calculate differences using broadcasting
    diffs = coord[:, np.newaxis, :] - coord[np.newaxis, :, :]
    squared_dists = np.sum(diffs ** 2, axis=2)

    # Extract the upper triangle indices where i < j
    i, j = np.triu_indices(len(coord), k=1)
    dist_arr = np.sqrt(squared_dists[i, j])
    dist_arr = dist_arr/min(dist_arr)
    return np.round(dist_arr, 2)

In [37]:
def B2T(Bextend):
    n = len(Bextend[0])
    Textend = np.zeros((n,n))

    for i in range(n):
        if Bextend[0][i]:
            Textend[i][i] = 1

    for B in Bextend:
        for i in range(n):
            for j in range(i+1,n):
                if B[i] and B[j]:
                    Textend[i][j] = 1
                    Textend[j][i] = 1
    return Textend.astype(int).tolist()

In [48]:
from pprint import pprint
import copy 

def exchange_rows(i, j, cp, mat):
    mat_copy = copy.deepcopy(mat)
    mat_copy[cp][-1] = 1
    mat_copy[i], mat_copy[j] = mat_copy[j], mat_copy[i]
    return mat_copy

def exchange_columns(i, j, mat, pos):
    mat_copy = copy.deepcopy(mat)
    if pos[i] == j:
        return mat_copy, pos
    if pos[j] != j:
        j = pos.index(j)
    for row_i in range(len(mat_copy)):
        mat_copy[row_i][i], mat_copy[row_i][j] = mat_copy[row_i][j], mat_copy[row_i][i]
    
    pos[i], pos[j] = pos[j], pos[i]
    return mat_copy, pos

#--- replace below code with info from 8bar.txt corresponding to the 8 bar mech type
# Read 8bar.txt and extract the section for the current mechanism

# Use the current type string for the mechanism (e.g., 'Type811')
mech_number = typesList[index].replace('Type', '').replace('-', '')
mech_name = f"Topo {mech_number}"
section_found = False
section_lines = []

with open('8bar.txt', 'r') as f:
    for line in f:
        if line.strip().startswith('Topo') and mech_name in line:
            section_found = True
            continue
        if section_found:
            if line.strip().startswith('Topo') and mech_name not in line:
                break  # End of current section
            if line.strip() and not line.strip().startswith('Topo'):
                section_lines.append(line)

if not section_lines:
    raise ValueError(f"Section for {mech_name} not found or empty in 8bar.txt.")

# Prepare a local namespace to exec the lines
local_vars = {}
exec(''.join(section_lines), {}, local_vars)

# Assign the extracted variables, with error handling
try:
    B = local_vars['B']
    grounds = local_vars['grounds']
    actuator = local_vars['actuator']
    actuator_fixed = local_vars['actuator_fixed']
    chain = local_vars['chain']
    coupler_point = local_vars['coupler_point']
except KeyError as e:
    raise KeyError(f"Variable {e} not found in section for {mech_name} in 8bar.txt. Check the file format.")


'''B = [[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]]


grounds = [0, 0, 1, 1, 5, 5, 6, 6, 2, 2, 2, 4, 4, 3, 3, 3]
actuator = [0, 2, 0, 1, 6, 7, 5, 4, 1, 3, 4, 6, 5, 2, 7, 3]
actuator_fixed = [2, 0, 1, 0, 7, 6, 4, 5, 3, 4, 1, 5, 6, 7, 3, 2]
chain = [1, [3, 7, 8], 2, [3, 4], [5, 9], [2, 3, 8], [6, 9], [1, 3], 0, [2, 7, 8], 5, 7, 4, 0, 6, [1, 4]]
coupler_point = [7, 6, 6, 7, 0, 1, 0, 7, 7, 7, 7, 0, 0, 6, 1, 6]'''

print(B)
print(grounds)
print(actuator)
print(actuator_fixed)
print(chain)
print(coupler_point)



[[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]]
[0, 0, 7, 2, 2, 2, 5, 5]
[0, 2, 9, 1, 3, 5, 9, 4]
[2, 0, 8, 3, 1, 3, 6, 6]
[1, [3, 4], [4, 6], 0, [2, 4], [7, 8], 8, [2, 3]]
[7, 7, 0, 7, 7, 7, 0, 1]


In [50]:
from shapely.geometry import LineString

#initStates = np.load("./npy-inputs/" + types[index] + '.npy')
initStates = np.load("/gpfs/scratch/raytang/walking-dataset-generator-old-simulator/Image-method-Synthesis-main/npy-inputs/RandPos-.npy")

NUM_MECHS = 5000

def is_curve_closed(coords, tolerance=0.1):
    """
    Check if a curve is closed by comparing the distance between first and last points.
    
    Parameters:
    - coords: numpy array of shape (N, 2) containing the curve points
    - tolerance: maximum distance between first and last points to consider as closed
    
    Returns:
    - Boolean: True if curve is closed, False otherwise
    """
    if len(coords) < 3:
        return False
    
    start_point = coords[0]
    end_point = coords[-1]
    distance = np.linalg.norm(end_point - start_point)
    
    return distance <= tolerance


def normalize_data_122223(X, scaling = 0, tol = 1e-8, maxiter = 2):
    X1, M1 = center_data(X) 
    X1, M2 = scale_data(X1, scaling = scaling)
    X1, M3 = rotate_data(X1)
    X1, M4 = reflect_data(X1)
    M = M4*M3*M2*M1 # This is the transformation matrix 

    detVal = np.abs(np.linalg.det(M))
    if detVal*scaling < tol:
        for i in range(maxiter):
            X1, M1 = center_data(X1)
            X1, M2 = scale_data(X1, scaling = scaling)
            X1, M3 = rotate_data(X1, randinit= True)
            X1, M4 = reflect_data(X1)
            if np.abs(np.linalg.det(M)) > tol or detVal*10 < np.abs(np.linalg.det(M)):
                break
    return X1, M4*M3*M2*M1, np.abs(np.linalg.det(M)) > tol

from scipy.ndimage import gaussian_filter1d
def has_sharp_edges(coords, curvature_threshold=15.0, smoothing_sigma=1.0):
    if smoothing_sigma > 0:
        x = gaussian_filter1d(coords[:,0], sigma=smoothing_sigma)
        y = gaussian_filter1d(coords[:,1], sigma=smoothing_sigma)
    else:
        x = coords[:,0]
        y = coords[:,1]

    dx = np.gradient(x)
    dy = np.gradient(y)
    ddx = np.gradient(dx)
    ddy = np.gradient(dy)

    curvature = np.abs(dx * ddy - dy * ddx) / (dx**2 + dy**2)**1.5
    return np.any(curvature > curvature_threshold), curvature

def resample_uniform(points, M):
    dif = np.diff(points, axis=0)
    seglen = np.sqrt((dif**2).sum(axis=1))
    s = np.concatenate(([0], np.cumsum(seglen)))
    total = s[-1]
    if total == 0:
        return np.repeat(points[:1], M, axis=0)
    s_uniform = np.linspace(0, total, M)
    x = np.interp(s_uniform, s, points[:,0])
    y = np.interp(s_uniform, s, points[:,1])
    return np.column_stack((x, y))

def pca_align(points):
    pts = points - np.mean(points, axis=0)
    U, S, VT = np.linalg.svd(pts, full_matrices=False)
    pcs = VT.T
    rotated = pts @ pcs
    return rotated, pcs

def detect_flat_via_pca(points, K_keep=12, N_resample=512, eps_slope=0.0075, min_fraction=1/4):
    pts = resample_uniform(points, N_resample)
    rotated, pcs = pca_align(pts)
    dy = np.gradient(rotated[:,1])
    flat_mask = np.abs(dy) < eps_slope
    
    from itertools import groupby
    runs = [sum(1 for _ in g) for val,g in groupby(flat_mask) if val]
    max_run = max(runs) if runs else 0
    flat_fraction = max_run / len(flat_mask)
    is_flat_enough = flat_fraction >= min_fraction
    
    first_pc = pcs[:,0]
    angle = np.arctan2(first_pc[1], first_pc[0])

    max_run = 0
    run_indices = []
    current_run = []

    for i, is_flat in enumerate(flat_mask):
        if is_flat:
            current_run.append(i)
            if len(current_run) > max_run:
                max_run = len(current_run)
                run_indices = current_run.copy()
        else:
            current_run = []

    return {
        'flat_fraction': float(flat_fraction),
        'is_flat_enough': bool(is_flat_enough),
        'orientation_rad': float(angle),
        'orientation_deg': float(np.degrees(angle)),
        'max_run': int(max_run),
        'N': int(len(flat_mask)),
        'flat_indices': np.array(run_indices, dtype=int), 
        'pts': pts,
    }

def ground_clearance_check(path_coords, joint_coords, flat_indices):
    if len(flat_indices) == 0:
        return False
    flat_pts = path_coords[flat_indices]
    m, b = np.polyfit(flat_pts[:,0], flat_pts[:,1], 1)

    d_path = path_coords[:,1] - (m*path_coords[:,0] + b)
    majority_sign = np.sign(np.sum(d_path))
    if majority_sign == 0:
        majority_sign = 1

    d_joints = joint_coords[:,1] - (m*joint_coords[:,0] + b)
    all_clear = np.all(np.sign(d_joints) == majority_sign)
    return all_clear

def save_flat_fit_with_joints(path_coords, flat_indices, joint_coords, save_dir, filename):
    if len(flat_indices) == 0:
        return
    
    flat_pts = path_coords[flat_indices]
    m, b = np.polyfit(flat_pts[:,0], flat_pts[:,1], 1)
    x_line = np.linspace(path_coords[:,0].min(), path_coords[:,0].max(), 200)
    y_line = m * x_line + b

    d_path = path_coords[:,1] - (m*path_coords[:,0] + b)
    majority_sign = np.sign(np.sum(d_path))
    if majority_sign == 0:
        majority_sign = 1

    d_joints = joint_coords[:,1] - (m*joint_coords[:,0] + b)

    plt.figure(figsize=(7,7))
    plt.plot(path_coords[:,0], path_coords[:,1], 'b-', label='Full curve')
    plt.plot(flat_pts[:,0], flat_pts[:,1], 'ro', label='Flat segment')
    plt.plot(x_line, y_line, 'k--', label='Best fit line (flat)')

    for i, (x, y) in enumerate(joint_coords):
        if np.sign(d_joints[i]) == majority_sign:
            plt.scatter(x, y, c='g', marker='x', s=80)
        else:
            plt.scatter(x, y, c='r', marker='x', s=80)
        plt.text(x, y, str(i), fontsize=9, ha='right', va='bottom')

    plt.axis('equal')
    plt.legend()
    plt.title(f"coords:{filename}")

    os.makedirs(save_dir, exist_ok=True)
    filename = f"{filename}.jpg"
    filepath = os.path.join(save_dir, filename)
    plt.savefig(filepath, format='jpg')
    plt.close()

def has_self_intersections(coords):
    line = LineString(coords)
    return not line.is_simple

def is_closed(pts):
    start_pt = pts[0]
    end_pt = pts[-1]
    path_dist = np.linalg.norm(end_pt - start_pt)
    is_closed = path_dist < 0.1
    return is_closed

def str_to_coords(s):
    s = str(s)
    nums = [float(x) for x in s.strip().split()]
    return [[nums[j], nums[j+1]] for j in range(0, len(nums), 2)]

# Worker function for parallel processing
for index in range(0, 16):
    mechType = index
    types   = [typesList[mechType]]
    print(f"Processing mechanism type: {typesList[mechType]}")
    
    couplerCurveIndex = 10
    errCtr = 0
    mechType = typesList[index]
    batch = []
    batchSaveStr = []
    batchSaveNpyStr = []
    
    for i, (g, a, af, c, cp) in enumerate(zip(grounds, actuator, actuator_fixed, chain, coupler_point)):
            
        B_new = exchange_rows(0, g, cp, B)
        col_positions = list(range(len(B[0])))
        
        B_new0, col_positions0 = exchange_columns(0, a, B_new, col_positions)
        B_new1, col_positions1 = exchange_columns(2, af, B_new0, col_positions0)
        B_new2, solSteps, c_final = None, None, None
        
        if type(c) == list:
            isOptim = False
            for ci in c:
                B_new2, rand = exchange_columns(1, ci, B_new1, col_positions1)
    
                _, solSteps, _ = computeSolSteps(linkMajor(B_new2))
                for solst in solSteps:
                    if solst[1] == 'optim':
                        isOptim = True
                        break
                c_final = ci
                if not isOptim:
                    break
                        
        else:
            B_new2, rand = exchange_columns(1, c, B_new1, col_positions1)
            _, solSteps, _ = computeSolSteps(linkMajor(B_new2))
            c_final = c
    
    
        T = B2T(B_new2)
        #pprint(B_new2)
        #print(mechType + str(i))
        #pprint(solSteps)
        


        saveDir = os.path.abspath("/gpfs/scratch/raytang/outputs-8bar/" + typesList[index] + str(i))
        saveDirNpy = os.path.abspath("/gpfs/scratch/raytang/outputs-8bar/" + typesList[index] + str(i) + "-npy")
        saveDirWalking = os.path.abspath("/gpfs/scratch/raytang/outputs-8bar/walking_" + typesList[index])
        saveDirWalkingPlot = os.path.abspath("/gpfs/scratch/raytang/outputs-8bar/walking_plot_" + typesList[index])
        
        # ADD THESE LINES TO CREATE DIRECTORIES:
        os.makedirs(saveDir, exist_ok=True)
        os.makedirs(saveDirNpy, exist_ok=True)
        os.makedirs(saveDirWalking, exist_ok=True)  # This is the missing one!
        os.makedirs(saveDirWalkingPlot, exist_ok=True)
            
        distStore = [np.zeros(int(55))]
    
        if not os.path.exists(saveDir):
            os.mkdir(saveDir)
    
        if not os.path.exists(saveDirNpy):
            os.mkdir(saveDirNpy)
    
        #for initState in tqdm(initStates):
        for initState in tqdm(initStates[:NUM_MECHS]):
            coord = np.round(initState, 3).reshape((int(22/2),2))
            dist = calc_dist(coord)
    
            if max(dist) > 10:
                continue
    
            if np.any(np.all(dist == distStore, axis=1)):
                continue
    
            distStore.append(dist)
            param = coord.tolist()
            name = str(param).replace("[", "").replace("]", "").replace(",", "")
    
            exampleData = {
                'T': T, 
                'solSteps': solSteps, 
                'params': param,
                'speedScale':speedscale, # 1 
                'steps':steps, # 360 
                'relativeTolerance':0.1 
            }
    
            # The transformation 
            #np.save(saveDir + name + ' ' + types[index], param)
    
            batch.append(exampleData)
            batchSaveStr.append(saveDir + '/' + name + ' ' + typesList[index] + str(i) + ' ')
            batchSaveNpyStr.append(saveDirNpy + '/' + name + ' ' + typesList[index] + str(i) + ' ')
    
            if len(batch) >= batchCount:
                #print(batch[0], '\n', batch[1])
                #print(batchSaveStr[0], '\n', batchSaveStr[1])
                try:
                    temp = requests.post(url = API_ENDPOINT, headers=HEADERS, data = json.dumps(batch)).json()
                    time.sleep(0.02)
                except ValueError as v:
                    for i in range(3):
                        time.sleep(2)
                        try:
                            temp = requests.post(url = API_ENDPOINT, headers=HEADERS, data = json.dumps(batch)).json()
                            break
                        except ValueError as v2:
                            errCtr += 1
                for i in range(len(temp)):
                    P = np.array(temp[i]['poses'])
                    np.save(batchSaveNpyStr[i] + '.npy', P)
                    #reak
                    try:
                        if len(P.shape) >= 1:
                            if P.shape[0] >= minsteps:
                                # do normalization, also get the transformation parameters. 
                                # also the paras are saved instead of MP (M: tranformation matrix, P: points in the matrix)
                                # This is just to avoid decimal difference problem 
                                imageMat, transParamSet, success = process_mech_102723(P, 10)
                                if success:
                                    #Tstr = np.array2string(np.round(transParamSet, 3), precision=3, suppress_small=True).replace("[", "").replace("]", "") # Wei asked for this part
                                    #Tstr = re.sub('\s+', ' ', Tstr) # to replace multiple sequential spaces together
                                    #binary_data = np.uint8(imageMat[:,:,0]) * 255
                                    #img = Image.fromarray(binary_data)
                                    #img.save(batchSaveStr[i] + Tstr + '.jpg')
                                    #plt.imshow(imageMat)
                                    #plt.savefig(batchSaveStr[i] + Tstr + '.jpg')
                                    #plt.clf()
                                    normalized_coords = normalize_data_122223(P[:,10,:], scaling=3.5)[0]
                    
                                    pca_output = detect_flat_via_pca(normalized_coords)
                                    sharp, kappa = has_sharp_edges(normalized_coords, curvature_threshold=150.0)
                                    self_intersecting = has_self_intersections(resample_uniform(normalized_coords, 512))
                                    
                                    joint_coords = np.array(str_to_coords(name))
                                    ground_clearance = ground_clearance_check(
                                        resample_uniform(P[:,10,:], 512),
                                        joint_coords,
                                        pca_output['flat_indices']
                                    )

                                    is_closed_curve = is_curve_closed(normalized_coords, tolerance=0.1)

                    
                                    # Check walking criteria
                                    if (pca_output['is_flat_enough'] and is_closed(normalized_coords) and 
                                        (not sharp) and ground_clearance and (not self_intersecting) and is_closed_curve):
                                        
                                        Tstr = np.array2string(np.round(transParamSet, 3), precision=3, suppress_small=True).replace("[", "").replace("]", "")
                                        Tstr = re.sub('\s+', ' ', Tstr)
                                        binary_data = np.uint8(imageMat[:,:,0]) * 255
                                        img = Image.fromarray(binary_data)
                    
                                        # Save walking mechanism
                                        img.save(saveDirWalking + "/" + name + ' ' + typesList[index] + ' ' + Tstr + '.jpg')
                                        save_flat_fit_with_joints(
                                            resample_uniform(P[:,10,:], 512), 
                                            pca_output['flat_indices'], 
                                            joint_coords, 
                                            saveDirWalkingPlot, 
                                            f"{name} {Tstr}"
                                        )
                                        print(f"savedwalking to: {str(saveDirWalking + '/' + name + ' ' + typesList[index] + ' ' + Tstr + '.jpg')}")
                                        
                                        result = {
                                            'type': typesList[index],
                                            'params': param,
                                            'flat_fraction': pca_output['flat_fraction'],
                                            'max_run': pca_output['max_run'],
                                            'orientation_deg': pca_output['orientation_deg'],
                                            'sharp': sharp,
                                            'ground_clearance': ground_clearance
                                        }
    
    
                    except ValueError as v:
                        print(v)
                    except FileNotFoundError as f:
                        print(f)
                batch = []
                batchSaveStr = []
                batchSaveNpyStr = []
    
        if len(batch) >= batchCount:
            #print(batch[0], '\n', batch[1])
            #print(batchSaveStr[0], '\n', batchSaveStr[1])
            try:
                temp = requests.post(url = API_ENDPOINT, headers=HEADERS, data = json.dumps(batch)).json()
                time.sleep(0.02)
            except ValueError as v:
                for i in range(3):
                    time.sleep(2)
                    try:
                        temp = requests.post(url = API_ENDPOINT, headers=HEADERS, data = json.dumps(batch)).json()
                        break
                    except ValueError as v2:
                            errCtr += 1
            for i in range(len(temp)):
                P = np.array(temp[i]['poses']) 
                np.save(batchSaveNpyStr[i] + '.npy', P)
                #print(P.shape)
                #reak
                try:
                    if len(P.shape) >= 1:
                        if P.shape[0] >= minsteps:
                            # do normalization, also get the transformation parameters. 
                            # also the paras are saved instead of MP (M: tranformation matrix, P: points in the matrix)
                            # This is just to avoid decimal difference problem 
                            imageMat, transParamSet, success = process_mech_102723(P, couplerCurveIndex)
                            if success:
    
                                normalized_coords = normalize_data_122223(P[:,10,:], scaling=3.5)[0]
                    
                                pca_output = detect_flat_via_pca(normalized_coords)
                                sharp, kappa = has_sharp_edges(normalized_coords, curvature_threshold=150.0)
                                self_intersecting = has_self_intersections(resample_uniform(normalized_coords, 512))
                                
                                joint_coords = np.array(str_to_coords(name))
                                ground_clearance = ground_clearance_check(
                                    resample_uniform(P[:, 10 ,:], 512),
                                    joint_coords,
                                    pca_output['flat_indices']
                                )

                                is_closed_curve = is_curve_closed(normalized_coords, tolerance=0.1)
                
                                # Check walking criteria
                                if (pca_output['is_flat_enough'] and is_closed(normalized_coords) and 
                                    (not sharp) and ground_clearance and (not self_intersecting) and is_closed_curve):
                                    
                                    Tstr = np.array2string(np.round(transParamSet, 3), precision=3, suppress_small=True).replace("[", "").replace("]", "")
                                    Tstr = re.sub('\s+', ' ', Tstr)
                                    binary_data = np.uint8(imageMat[:,:,0]) * 255
                                    img = Image.fromarray(binary_data)
                
                                    # Save walking mechanism
                                    img.save(saveDirWalking + "/" + name + ' ' + typesList[index] + ' ' + Tstr + '.jpg')
                                    save_flat_fit_with_joints(
                                        resample_uniform(P[:,10,:], 512), 
                                        pca_output['flat_indices'], 
                                        joint_coords, 
                                        saveDirWalkingPlot, 
                                        f"{name} {Tstr}"
                                    )
                                    print(f"savedwalking to: {str(saveDirWalking + '/' + name + ' ' + typesList[index] + ' ' + Tstr + '.jpg')}")
                                    
                                    result = {
                                        'type': typesList[mechType],
                                        'params': param,
                                        'flat_fraction': pca_output['flat_fraction'],
                                        'max_run': pca_output['max_run'],
                                        'orientation_deg': pca_output['orientation_deg'],
                                        'sharp': sharp,
                                        'ground_clearance': ground_clearance
                                    }
                except ValueError as v:
                    print(v)
                except FileNotFoundError as f:
                    print(f)
            batch = []
            batchSaveStr = []
            batchSaveNpyStr = []
    


Processing mechanism type: Type811-


100%|██████████| 5000/5000 [00:50<00:00, 99.31it/s] 


Processing mechanism type: Type812-


100%|██████████| 5000/5000 [00:50<00:00, 99.91it/s] 


Processing mechanism type: Type813-


 14%|█▎        | 685/5000 [00:06<00:43, 98.16it/s] 


KeyboardInterrupt: 


KeyboardInterrupt

